# Spring 2022
# CPSC 585 Project 4
## Raymond Carpio
## Yu Pan
## Sijie Shang
## John Tu

### Based on the example from TensorFlow's RNN text generation:
### https://www.tensorflow.org/text/tutorials/text_generation

In [1]:
# Import the following modules over
import numpy as np
import tensorflow as tf
import os
import time

# Try to open the text file if it exists.
input_file = "infinity-war.txt"
corpus_raw = open(input_file, "r", encoding="utf-8").read()

# Print out the sample text file.
#print(corpus_raw)

In [2]:
# Obtain the list of characters included in the raw text.
# Be sure to use set() to filter out duplicates.
characters = sorted(list(set(corpus_raw)))

# Print out the list of characters found in the raw text.
print(characters)

# Also obtain the total length of the text and the characters.
print("Total length of text: ", len(corpus_raw))
print("Total number of characters found: ", len(characters))

['\n', ' ', '!', '"', '%', '&', "'", ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '’', '…']
Total length of text:  60487
Total number of characters found:  77


In [3]:
# Print out the first 500 characters of the text.
print(corpus_raw[:500])





Asgardian PA: This is the Asgardian refugee vessel Statesman. We are under assault, I repeat, we are under assault - The engines are dead, life support failing. Requesting aid from any vessel within range. We are 22 jump points out of Asgard.  Our crew is made up of Asgardian families, we have very few soldiers here. This is not a warcraft. I repeat, this is not a warcraft!



Ebony Maw: Hear me, and rejoice. You have had the privilege of being saved by the Great Titan... You may think this 


In [4]:
# Vectorize the text by converting the string into a numerical form.
sample_text = ['abcdefghij', 'wxyz']
num_chars = tf.strings.unicode_split(sample_text, input_encoding="UTF-8")
num_chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g', b'h', b'i', b'j'],
 [b'w', b'x', b'y', b'z']]>

In [5]:
id_from_characters = tf.keras.layers.StringLookup(vocabulary=list(characters))
ids = id_from_characters(num_chars)
ids

<tf.RaggedTensor [[50, 51, 52, 53, 54, 55, 56, 57, 58, 59], [72, 73, 74, 75]]>

In [6]:
characters_from_id = tf.keras.layers.StringLookup(vocabulary=id_from_characters.get_vocabulary(), invert=True)
characters = characters_from_id(ids)
characters

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g', b'h', b'i', b'j'],
 [b'w', b'x', b'y', b'z']]>

In [7]:
# Convert the corresponding string ids back into text form.
def text_from_id(ids):
    return tf.strings.reduce_join(characters_from_id(ids), axis=-1)

In [8]:
all_text_id = id_from_characters(tf.strings.unicode_split(corpus_raw, 'UTF-8'))
all_text_id

<tf.Tensor: shape=(60487,), dtype=int64, numpy=array([ 1,  1,  1, ..., 10,  1,  1], dtype=int64)>

In [9]:
# Try to print out the first 25 characters from raw text via tensor slices.
id_dataset = tf.data.Dataset.from_tensor_slices(all_text_id)
for curr_id in id_dataset.take(25):
    print(characters_from_id(curr_id).numpy().decode('utf-8'))









A
s
g
a
r
d
i
a
n
 
P
A
:
 
T
h
i
s
 
i
s


In [10]:
# Now build a sequence of the first 150 characters.
sequence_len = 150
each_example = len(corpus_raw)
sequences = id_dataset.batch(sequence_len+1, drop_remainder=True)

for curr_seq in sequences.take(1):
    print(characters_from_id(curr_seq))

tf.Tensor(
[b'\n' b'\n' b'\n' b'\n' b'A' b's' b'g' b'a' b'r' b'd' b'i' b'a' b'n' b' '
 b'P' b'A' b':' b' ' b'T' b'h' b'i' b's' b' ' b'i' b's' b' ' b't' b'h'
 b'e' b' ' b'A' b's' b'g' b'a' b'r' b'd' b'i' b'a' b'n' b' ' b'r' b'e'
 b'f' b'u' b'g' b'e' b'e' b' ' b'v' b'e' b's' b's' b'e' b'l' b' ' b'S'
 b't' b'a' b't' b'e' b's' b'm' b'a' b'n' b'.' b' ' b'W' b'e' b' ' b'a'
 b'r' b'e' b' ' b'u' b'n' b'd' b'e' b'r' b' ' b'a' b's' b's' b'a' b'u'
 b'l' b't' b',' b' ' b'I' b' ' b'r' b'e' b'p' b'e' b'a' b't' b',' b' '
 b'w' b'e' b' ' b'a' b'r' b'e' b' ' b'u' b'n' b'd' b'e' b'r' b' ' b'a'
 b's' b's' b'a' b'u' b'l' b't' b' ' b'-' b' ' b'T' b'h' b'e' b' ' b'e'
 b'n' b'g' b'i' b'n' b'e' b's' b' ' b'a' b'r' b'e' b' ' b'd' b'e' b'a'
 b'd' b',' b' ' b'l' b'i' b'f' b'e' b' ' b's' b'u' b'p'], shape=(151,), dtype=string)


In [11]:
for curr_seq in sequences.take(5):
    print(text_from_id(curr_seq).numpy())

b'\n\n\n\nAsgardian PA: This is the Asgardian refugee vessel Statesman. We are under assault, I repeat, we are under assault - The engines are dead, life sup'
b'port failing. Requesting aid from any vessel within range. We are 22 jump points out of Asgard.  Our crew is made up of Asgardian families, we have ver'
b'y few soldiers here. This is not a warcraft. I repeat, this is not a warcraft!\n\n\n\nEbony Maw: Hear me, and rejoice. You have had the privilege of being '
b'saved by the Great Titan... You may think this is suffering... No. it is salvation. Universal scales tip toward balance because of your sacrifice. Smil'
b"e...  For even in death, you have become Children of Thanos.\n\n\n\nThanos:  I know what it's like to lose. To feel so desperately that you're right... yet"


In [12]:
def split_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [13]:
text_dataset = sequences.map(split_target)
for input_text, target_text in text_dataset.take(1):
    print("Input: ", text_from_id(input_text).numpy())
    print("Target: ", text_from_id(target_text).numpy())

Input:  b'\n\n\n\nAsgardian PA: This is the Asgardian refugee vessel Statesman. We are under assault, I repeat, we are under assault - The engines are dead, life su'
Target:  b'\n\n\nAsgardian PA: This is the Asgardian refugee vessel Statesman. We are under assault, I repeat, we are under assault - The engines are dead, life sup'


In [14]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    text_dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 150), dtype=tf.int64, name=None), TensorSpec(shape=(64, 150), dtype=tf.int64, name=None))>

In [15]:
vocab = sorted(set(corpus_raw))
vocab_size = len(vocab)
embedding_dimension=256
rnn_units=1024

In [16]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [17]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(id_from_characters.get_vocabulary()),
    embedding_dim=embedding_dimension,
    rnn_units=rnn_units)

In [18]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 150, 78) # (batch_size, sequence_length, vocab_size)


In [19]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  19968     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  79950     
                                                                 
Total params: 4,038,222
Trainable params: 4,038,222
Non-trainable params: 0
_________________________________________________________________


In [20]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices

array([ 1, 24, 19, 13, 48, 16, 70, 66, 74, 22, 57, 14, 34, 38, 39, 59, 12,
       33, 53, 75, 39, 70, 11, 55, 37, 28, 45, 63, 71, 68, 64, 26, 18, 29,
       54, 55,  6, 54,  4, 72, 22, 52, 17,  0, 53, 34, 52, 49, 70, 38, 11,
       71, 63, 16, 36,  3,  3, 48, 36,  3, 24,  0, 27, 40,  5, 31, 59, 76,
       68, 61, 20, 74, 53,  0, 49, 66, 55, 10, 10, 57, 28, 27, 11, 40, 61,
       48, 73, 34, 60, 27, 54, 29, 21, 16, 47,  0,  5, 54, 56, 20, 17, 39,
       49,  0, 20, 28, 27, 49,  4, 59,  1, 57, 58, 37, 74, 49, 55, 67, 12,
       13, 25, 71,  2, 29,  0,  4, 69, 61, 69, 55, 51, 50, 53, 44, 31, 13,
       37, 16, 49, 56, 48, 16, 40, 61, 52, 41, 71, 27,  8, 16],
      dtype=int64)

In [21]:
print("Input:\n", text_from_id(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_id(sampled_indices).numpy())

Input:
 b' these dwarves are better at forging than they are cleaning. Maybe they realized they live in a junk pile in the middle of space.\n\nThor: This forge ha'

Next Char Predictions:
 b'\nB71Z4uqy?h2LPQj0KdzQu.fOFWnvsoD6Gef&e"w?c5[UNK]dLc]uP.vn4N!!ZN!B[UNK]ER%Ij\xe2\x80\x99sl8yd[UNK]]qf--hFE.RlZxLkEeG:4Y[UNK]%eg85Q][UNK]8FE]"j\nhiOy]fr01Cv G[UNK]"tltfbadVI1O4]gZ4RlcSvE)4'


In [22]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)
tf.exp(example_batch_mean_loss).numpy()

Prediction shape:  (64, 150, 78)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.356975, shape=(), dtype=float32)


78.02077

In [23]:
model.compile(optimizer='adam', loss=loss)

In [24]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
history = model.fit(dataset, epochs=30, callbacks=[checkpoint_callback])

Epoch 1/30
6/6 [==============================] - 22s 3s/step - loss: 4.9595
Epoch 2/30
6/6 [==============================] - 20s 3s/step - loss: 4.1037
Epoch 3/30
6/6 [==============================] - 21s 4s/step - loss: 3.9537
Epoch 4/30
6/6 [==============================] - 24s 4s/step - loss: 3.5004
Epoch 5/30
6/6 [==============================] - 25s 4s/step - loss: 3.2321
Epoch 6/30
6/6 [==============================] - 25s 4s/step - loss: 3.1338
Epoch 7/30
6/6 [==============================] - 24s 4s/step - loss: 3.0262
Epoch 8/30
6/6 [==============================] - 25s 4s/step - loss: 2.9156
Epoch 9/30
6/6 [==============================] - 23s 4s/step - loss: 2.7912
Epoch 10/30
6/6 [==============================] - 24s 4s/step - loss: 2.6704
Epoch 11/30
6/6 [==============================] - 24s 4s/step - loss: 2.5789
Epoch 12/30
6/6 [==============================] - 24s 4s/step - loss: 2.5107
Epoch 13/30
6/6 [==============================] - 26s 4s/step - loss: 2.

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, characters_from_id, id_from_characters, 1.0)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['This'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

## If there are modifications made for this code, then it would be the length of the example sequence for input text processing and the number of epochs to run for training. Also, the changes made so far are the example text vector and the temperature parameter for the one-step RNN model class.